In [1]:
import yfinance as yf
from pyspark.sql import SparkSession

from datetime import timedelta, datetime
from dateutil.relativedelta import relativedelta

In [2]:
# Functions
def get_start_date(input_str):
    """
    Find the start date by subtracting the given time range from "yesterday"
    :param input_str: string that represents time range: <num><y/m/d>
                      Example: 2y-2 years from yesterday;
                               2d-2 days from yesterday.
    :return: the start date calculated in datetime
    """
    unit = input_str[-1]  # Get the last character (y, m, d)
    quantity = int(input_str[:-1])  # Get the quantity as integer

    # Subtract the quantity of the specified unit from today's date
    if unit == 'y':
        return datetime.now() - relativedelta(years=quantity)
    elif unit == 'm':
        return datetime.now() - relativedelta(months=quantity)
    elif unit == 'd':
        return datetime.now() - relativedelta(days=quantity)
    else:
        raise ValueError("Invalid time unit. Use 'y' for years, 'm' for months, and 'd' for days.")

In [3]:
# Define the date range
date_range = '5y'

start_date = get_start_date(date_range)
end_date = datetime.now() - timedelta(days=1)

start_date_str = start_date.strftime('%Y-%m-%d')
end_date_str = end_date.strftime('%Y-%m-%d')

In [4]:
# Define the ticker symbol.
tickerSymbol = 'MSFT'

# Get the data of the ticker.
tickerData = yf.Ticker(tickerSymbol)

# Get the historical prices for this ticker
tickerDf = tickerData.history(start=start_date, end=end_date)

# Display the last few rows
print(tickerDf.tail())

                                 Open        High         Low       Close  \
Date                                                                        
2024-02-07 00:00:00-05:00  407.440002  414.299988  407.399994  414.049988   
2024-02-08 00:00:00-05:00  414.049988  415.559998  412.529999  414.109985   
2024-02-09 00:00:00-05:00  415.250000  420.820007  415.089996  420.549988   
2024-02-12 00:00:00-05:00  420.559998  420.739990  414.750000  415.260010   
2024-02-13 00:00:00-05:00  404.940002  410.070007  403.390015  406.320007   

                             Volume  Dividends  Stock Splits  
Date                                                          
2024-02-07 00:00:00-05:00  22340500        0.0           0.0  
2024-02-08 00:00:00-05:00  21225300        0.0           0.0  
2024-02-09 00:00:00-05:00  22012900        0.0           0.0  
2024-02-12 00:00:00-05:00  21202900        0.0           0.0  
2024-02-13 00:00:00-05:00  27824900        0.0           0.0  


C:\Users\henry\AppData\Local\Programs\Python\Python311\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


In [10]:
spark = SparkSession.builder \
    .master("local") \
    .appName("YourAppName") \
    .config("spark.memory.offHeap.enabled","true") \
    .config("spark.memory.offHeap.size","10g") \
    .getOrCreate()

sdf = spark.createDataFrame(tickerDf.reset_index())

In [11]:
sdf.show()

Py4JJavaError: An error occurred while calling o118.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 1 times, most recent failure: Lost task 0.0 in stage 2.0 (TID 2) (10.8.139.113 executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:364)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:328)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(